In [ ]:
#tweeter
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

ckey = "mXEtCJeGbCtpiQw0oXWY02Mil"
csecret = "j4w9WayaSu8YftCzaj2Tj7U4b8POw7wb9rfLaNwP2yRzBkYn67"
atoken = "716546342596820992-WumCCtJ586Gk4Ulj0ZEK86vLxNmWoB6"
asecret = "O7w8W48mUeJm96cBLfeJDtCAUXDB9muCdiPwP9yQl7eZ0"

#utils
import json
from datetime import datetime

'''========postgresql'=========='''
import psycopg2

conn = psycopg2.connect("dbname=data user=postgres password=naty")
cur = conn.cursor()

##create tabs
cur.execute("CREATE TABLE IF NOT EXISTS data.tweets (id varchar(600), created_at date, extended_tweet varchar(600));")

def save(tweet):
    _id,created_at,extended_tweet,timestamp_ms,favorite_count,retweet_count,user_mentions = tweet.values()
    query = "INSERT INTO data.tweets (id, created_at,extended_tweet) VALUES (%s,%s,%s)"
    params = (_id,created_at,extended_tweet)
    cur.execute(query,params)
    conn.commit()
    

def get_date(timestamp):
    date = datetime.fromtimestamp(timestamp/1000.0)
    date = date.strftime('%Y-%m-%d %H:%M:%S')
    return date;

class listener(StreamListener):
    
    def on_data(self, data):
        fields = json.loads(data)
        
        def validate(fields,key):
            if key in fields:
                return fields[key];
            else:
                return None     
        
            
        try:                
            #create data tuple/object/json
            
            tweet = {
                "_id" : validate(fields,"id_str") ,
                "created_at" : validate(fields,"created_at"),
                "extended_tweet": validate(fields,"extended_tweet")["full_text"] if validate(fields,"extended_tweet") != None else None,
                "timestamp_ms": get_date(int(validate(fields,"timestamp_ms"))) if validate(fields,"timestamp_ms")!= None else None,
                "favorite_count": validate(fields,"favorite_count"),
                "retweet_count": validate(fields,"retweet_count"),
                "user_mentions": len(validate(fields,"entities")["user_mentions"]) if validate(fields,"entities") != None else 0
            }
                        
            print ("_id :"+str(tweet["_id"])+"\n")
            print ("created_at :"+str(tweet["created_at"])+"\n")
            print ("timestamp_ms :"+str(tweet["timestamp_ms"])+"\n")
            print ("extended_tweet :"+str(tweet["extended_tweet"])+"\n")
            print ("favorite_count :"+str(tweet["favorite_count"])+"\n")
            print ("retweet_count :"+str(tweet["retweet_count"])+"\n")
            print ("user_mentions :"+str(tweet["user_mentions"])+"\n")
            
            save(tweet);
            print("saved!!")
            print("\n\n")

        except IndexError as e:
            print (e)
            pass
        return True
    
    def on_error(self, status):
        print (status)
        
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)
twitterStream = Stream(auth, listener())


twitterStream.filter(track=['covid']);